In [12]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from load_utils import *
import algo_utils

In [26]:
products_cat = load_obj('../data/products_table')
traffic_cat = load_obj('../data/traffic_table')
purchased_cat = load_obj('../data/purchased_table')
print("Loaded datsets")

Loaded datsets


In [27]:
products_cat.sort_values(by="ProductId")
products_cat_ids = products_cat["ProductId"].drop_duplicates().values
n = len(products_cat_ids)
# products_cat_ids has length 7797 this is the dimension of the state vector. 

In [73]:
def state_to_ids(state):
    products_cat.sort_values(by="ProductId")
    return(products_cat["ProductId"].drop_duplicates().values[state.astype(bool)])

def ids_to_state(ids):
    n = len(products_cat_ids)
    products_cat.sort_values(by="ProductId")
    state = np.zeros(n)
    state[np.where(products_cat["ProductId"].drop_duplicates().isin(ids))[0]]=1
    return(state)

def next_state(state, question, answer):
    ids = state_to_ids(state)
    product_set = products_cat.loc[products_cat["ProductId"].isin(ids),]
    if answer=='idk': # case i don't know the answer return everything
        return(state)
    else:
        q_keep = set(product_set.loc[product_set["PropertyDefinitionId"]==int(question), "ProductId"].index.values) # had to remove this drop_duplicates() because it changed the index !!!!!!!
        a_keep = set(product_set.loc[product_set["answer"].astype(str)==str(answer), "ProductId"].index.values)
        total = a_keep.intersection(q_keep)
        products_to_keep = product_set.loc[total, "ProductId"].drop_duplicates().values          
        state = ids_to_state(products_to_keep)       
        return(state)

In [62]:
def init_env():
    y = np.random.choice(products_cat["ProductId"].drop_duplicates().values, size = 1)[0]
    quest_answer_y = algo_utils.get_answers_y(y, products_cat)
    s = np.repeat(1, 7797) #all products in the initial state
    setA = list(set(algo_utils.get_questions(products_cat)))
    return(s, setA, quest_answer_y)

In [94]:
def play(state, action, answer, q_asked=[]):
    new_state = next_state(state, action, answer)
    if np.sum(new_state)<=50:
        print('finished the game')
        return(new_state, 100.0/len(q_asked))
    else:
        print('there is still {} product left'.format(np.sum(new_state)))
        return(new_state, - float(np.sum(new_state))/np.sum(state))
    

In [115]:
# totally random episode
D = []
capacity_D = 200
state, setA, quest_answer_y = init_env()
while (len(D)<capacity_D):
    state, setA, quest_answer_y = init_env()
    q_asked = []
    while((np.sum(state)>50) and len(setA)>0): #each run of this loop is one episode
        q = np.random.choice(setA, size=1)[0]
        setA = list(set(setA).difference(set([q])))
        q_asked.append(q)
        old_state = state
        state, r = play(state, q, quest_answer_y[q], q_asked)
        D.append((old_state, q, r, state))
        print(r)

there is still 3819.0 product left
-0.48980377068103115
there is still 3819.0 product left
-1.0
there is still 3819.0 product left
-1.0
there is still 3819.0 product left
-1.0
there is still 3819.0 product left
-1.0
there is still 3819.0 product left
-1.0
there is still 3819.0 product left
-1.0
there is still 3819.0 product left
-1.0
there is still 3819.0 product left
-1.0
there is still 3819.0 product left
-1.0
there is still 3819.0 product left
-1.0
there is still 3819.0 product left
-1.0
there is still 3819.0 product left
-1.0
there is still 3819.0 product left
-1.0
there is still 3819.0 product left
-1.0
there is still 868.0 product left
-0.22728462948415815
there is still 868.0 product left
-1.0
there is still 868.0 product left
-1.0
there is still 868.0 product left
-1.0
there is still 868.0 product left
-1.0
there is still 868.0 product left
-1.0
there is still 868.0 product left
-1.0
there is still 868.0 product left
-1.0
there is still 868.0 product left
-1.0
there is still 86

there is still 3895.0 product left
-1.0
there is still 3895.0 product left
-1.0
there is still 3895.0 product left
-1.0
there is still 3895.0 product left
-1.0
there is still 3895.0 product left
-1.0
there is still 3895.0 product left
-1.0
there is still 3895.0 product left
-1.0
there is still 3895.0 product left
-1.0
there is still 3895.0 product left
-1.0
there is still 3895.0 product left
-1.0
there is still 3895.0 product left
-1.0
there is still 3895.0 product left
-1.0
there is still 3895.0 product left
-1.0
there is still 2258.0 product left
-0.5797175866495508
there is still 2258.0 product left
-1.0
there is still 2258.0 product left
-1.0
there is still 2258.0 product left
-1.0
there is still 2258.0 product left
-1.0
there is still 2258.0 product left
-1.0
there is still 2258.0 product left
-1.0
there is still 2258.0 product left
-1.0
there is still 2258.0 product left
-1.0
there is still 2258.0 product left
-1.0
there is still 2258.0 product left
-1.0
there is still 2258.0 pro

In [97]:
#need to create dense netwrok
q_len = len(setA) 
questions_available = tf.placeholder([q_len])
# suppose we have state, q, r, new_state
n = len(products_cat_ids)
state = tf.placeholder([1, n])
pred = network_Q(state, action)
list_q = [network_Q(new_state, a) for a in q]
target =  r + gamma*max(list_q)
opt_network(loss=(pred-target)^2)

In [21]:
state[[1,3]]=1

In [39]:
def state_to_ids(state):
    return(products_cat["ProductId"].drop_duplicates().values[state.astype(bool)])

In [40]:
ids = state_to_ids(state)
products_cat.loc[products_cat["ProductId"].isin(ids),]

,ProductId,BrandId,ProductTypeId,PropertyValue,PropertyDefinitionId,PropertyDefinitionOptionId,answer
1,6300952,314,6,5.306000e+03,19219,NaN,5.306000e+03
3,6973019,314,6,3.259490e+05,19219,NaN,3.259490e+05
11010,6300952,314,6,3.379000e+03,10525,NaN,3.379000e+03
11546,6973019,314,6,3.379000e+03,10525,NaN,3.379000e+03
13823,6300952,314,6,3.379000e+03,10526,NaN,3.379000e+03
14767,6973019,314,6,3.379000e+03,10526,NaN,3.379000e+03
22357,6300952,314,6,1.660050e+05,11075,NaN,1.660050e+05
90183,6300952,314,6,1.000000e+00,13176,NaN,1.000000e+00
90881,6973019,314,6,1.000000e+00,13176,NaN,1.000000e+00
93120,6300952,314,6,8.000000e+00,13177,NaN,8.000000e+00


In [48]:
state = np.where(products_cat["ProductId"].drop_duplicates().isin(ids))[0]

In [50]:
state

array([1, 3])

In [51]:
def ids_to_state(ids):
    n = len(products_cat_ids)
    products_cat.sort_values(by="ProductId")
    state = np.zeros(n)
    state[np.where(products_cat["ProductId"].drop_duplicates().isin(ids))[0]]=1
    return(state)

In [52]:
ids_to_state(ids)

array([0., 1., 0., ..., 0., 0., 0.])